<a href="https://colab.research.google.com/github/nhnam1105/basket-market-analysis/blob/master/Market_basket_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 59 kB/s 
     |████████████████████████████████| 198 kB 63.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=afb908ee7691c69738f2b891d8530a45355de3a9d9bebe59c097214e8ad47fa6
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [ ]:
#Upload kaggle.json file
from google.colab import files

kaggle_file = files.upload()
!mkdir /root/.kaggle
!mv kaggle.json /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
import kaggle
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import zipfile
import pandas as pd
import numpy as np
import tweepy
from kaggle.api.kaggle_api_extended import KaggleApi
from pyspark import Row
from pyspark.sql.types import StructType, StructField, LongType
from pyspark.sql import functions as F
from pyspark.sql.functions import split, col, collect_set
from itertools import combinations, product
from collections import defaultdict
import time

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
#Create new Spark Context and configuration
conf = SparkConf().setAppName("MarketBasketAnalysis")
conf = (conf.setMaster('local[*]')
        .set('spark.executor.memory', '8G')
        .set('spark.driver.memory', '45G')
        .set('spark.driver.maxResultSize', '0')
        .set("spark.sql.execution.arrow.enabled", "true"))
sc = SparkContext.getOrCreate(conf=conf)
spark = SQLContext(sc)

In [ ]:
#Define Kaggle api to download data
api = KaggleApi()
api.authenticate()

In [ ]:
!kaggle datasets download -d ashirwadsangwan/imdb-dataset
!unzip imdb-dataset.zip

100% 1.44G/1.44G [00:22<00:00, 91.8MB/s]
100% 1.44G/1.44G [00:22<00:00, 70.1MB/s]
Archive:  imdb-dataset.zip
  inflating: name.basics.tsv.gz      
  inflating: name.basics.tsv/name.basics.tsv  
  inflating: title.akas.tsv.gz       
  inflating: title.akas.tsv/title.akas.tsv  
  inflating: title.basics.tsv.gz     
  inflating: title.basics.tsv/title.basics.tsv  
  inflating: title.principals.tsv.gz  
  inflating: title.principals.tsv/title.principals.tsv  
  inflating: title.ratings.tsv.gz    
  inflating: title.ratings.tsv/title.ratings.tsv  


# Exploratory Data Analysis

In [ ]:
#Read csv file to spark
castCrew = spark.read.csv("/content/name.basics.tsv", sep=r'\t', header=True, inferSchema=True)
movies = spark.read.csv("/content/title.basics.tsv", sep=r'\t', header=True, inferSchema=True)
principals = spark.read.csv("/content/title.principals.tsv", sep=r'\t', header=True, inferSchema=True)

In [ ]:
castCrew.show(5)

+---------+---------------+---------+---------+--------------------+--------------------+
|   nconst|    primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+---------------+---------+---------+--------------------+--------------------+
|nm0000001|   Fred Astaire|     1899|     1987|soundtrack,actor,...|tt0050419,tt00531...|
|nm0000002|  Lauren Bacall|     1924|     2014|  actress,soundtrack|tt0071877,tt01170...|
|nm0000003|Brigitte Bardot|     1934|       \N|actress,soundtrac...|tt0054452,tt00491...|
|nm0000004|   John Belushi|     1949|     1982|actor,writer,soun...|tt0077975,tt00725...|
|nm0000005| Ingmar Bergman|     1918|     2007|writer,director,a...|tt0069467,tt00509...|
+---------+---------------+---------+---------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
movies.show(5)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            \N|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

In [ ]:
principals.show(5)

+---------+--------+---------+---------------+--------------------+-----------+
|   tconst|ordering|   nconst|       category|                 job| characters|
+---------+--------+---------+---------------+--------------------+-----------+
|tt0000001|       1|nm1588970|           self|                  \N|["Herself"]|
|tt0000001|       2|nm0005690|       director|                  \N|         \N|
|tt0000001|       3|nm0374658|cinematographer|director of photo...|         \N|
|tt0000002|       1|nm0721526|       director|                  \N|         \N|
|tt0000002|       2|nm1335271|       composer|                  \N|         \N|
+---------+--------+---------+---------------+--------------------+-----------+
only showing top 5 rows



CreateOrReplaceTempView will create a temporary view of the table on memory it is not persistent at this moment, we can run SQL query on top of that.

In [ ]:
castCrew.createOrReplaceTempView("castCrew")
principals.createOrReplaceTempView("principals")
movies.createOrReplaceTempView("movies")

In [ ]:
#From movie table we filter with Movie only using SQL syntax
movie_query = '''
  SELECT *
  From movies
  WHERE titleType = 'movie'
 ''' 

In [ ]:
movies = spark.sql(movie_query)
movies.show(5)
movies.createOrReplaceTempView('movies')

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000009|    movie|          Miss Jerry|          Miss Jerry|      0|     1894|     \N|            45|             Romance|
|tt0000147|    movie|The Corbett-Fitzs...|The Corbett-Fitzs...|      0|     1897|     \N|            20|Documentary,News,...|
|tt0000335|    movie|Soldiers of the C...|Soldiers of the C...|      0|     1900|     \N|            \N|     Biography,Drama|
|tt0000502|    movie|            Bohemios|            Bohemios|      0|     1905|     \N|           100|                  \N|
|tt0000574|    movie|The Story of the ...|The Story of the ...|      0|     1906|     \N|            70|Biography,Crim

In [ ]:
#From Principals table we filter with actor/actress using SQL syntax
characters_query = '''
  SELECT *
  FROM principals
  WHERE (category = 'actor') OR (category = 'actress')
 '''

In [ ]:
principals = spark.sql(characters_query)
principals.show(5)
principals.createOrReplaceTempView('principals')

+---------+--------+---------+--------+---+----------------+
|   tconst|ordering|   nconst|category|job|      characters|
+---------+--------+---------+--------+---+----------------+
|tt0000005|       1|nm0443482|   actor| \N|  ["Blacksmith"]|
|tt0000005|       2|nm0653042|   actor| \N|   ["Assistant"]|
|tt0000007|       1|nm0179163|   actor| \N|              \N|
|tt0000007|       2|nm0183947|   actor| \N|              \N|
|tt0000008|       1|nm0653028|   actor| \N|["Sneezing Man"]|
+---------+--------+---------+--------+---+----------------+
only showing top 5 rows



We now retrieve table that contains tconst and nconst \\
LEFT JOIN returns all the rows of the table on the left side of the join and matching rows for the table on the right side of join. The rows for which there is no matching row on right side, the result-set will contain null

In [ ]:
query =  """
SELECT p.tconst, p.nconst
            FROM principals AS p LEFT JOIN movies as m ON p.tconst == m.tconst                       
            WHERE (p.category == 'actor' OR p.category == 'actress')  AND m.titleType == 'movie'
"""

In [ ]:
res = spark.sql(query)
baskets = res.groupBy('tconst').agg(collect_set('nconst').alias('items'))

In [ ]:
baskets.show(5, False)

+---------+---------------------------------------------------------------------------------------------------+
|tconst   |items                                                                                              |
+---------+---------------------------------------------------------------------------------------------------+
|tt0002591|[nm0029806, nm0509573]                                                                             |
|tt0003689|[nm0910564, nm0527801, nm0399988, nm0101071, nm0694718, nm0728289, nm0585503]                      |
|tt0004272|[nm0368875, nm0092665, nm0492302, nm0445507, nm0776747, nm0383278, nm0192062, nm0285643, nm0793189]|
|tt0004336|[nm0268437, nm0811293, nm0102718, nm0190516, nm0478359]                                            |
|tt0005209|[nm0593671, nm0394389, nm0203439, nm0909648, nm0863863, nm0269671]                                 |
+---------+---------------------------------------------------------------------------------------------

In [ ]:
baskets.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- items: array (nullable = false)
 |    |-- element: string (containsNull = false)



# A-Priori

In [ ]:
indexed_rdd = baskets.select('items').rdd.flatMap(list)

In [ ]:
count = indexed_rdd.count()

In [ ]:
count

393656

In [ ]:
index_flatted = indexed_rdd.flatMap(list)

In [ ]:
index_flatted.take(5)

['nm0161366', 'nm0865178', 'nm0001908', 'nm0607104', 'nm0639631']

In [ ]:
def create_singletons (rdd): 
  #generates a list of actors, map them to 1 to initialize the counter for each 
  #actor and update the counter for every occurrence
  singletons = rdd.flatMap(list) \
                  .map(lambda item: (item, 1)) \
                  .reduceByKey(lambda x, y: x+y)
  return(singletons)

In [ ]:
singleton = indexed_rdd.map(lambda item: (item, 1)).reduceByKey(lambda a,b:a +b).filter(lambda item: item[1] >= 150).map(lambda item: (item[0]))

In [ ]:
singleton = create_singletons(indexed_rdd) \
              .filter(lambda item: item[1] >= 140) \
              .map(lambda item: (item[0])) 

In [ ]:
singleton.count()

268

In [ ]:
def getFrequentItemsets(items, c):
  items = items.map(lambda item: list(item[0])) 
  #generate singletons
  singleton = create_singletons(items).map(lambda item: [item[0]]) 
  #create an empty list that contain the new possible combinations
  items_set = [] 
  for comb in list(product(items.toLocalIterator(), singleton.toLocalIterator())):
    comb = comb[0]+comb[1] 
    comb_set = sorted(set().union(comb))
    if (len(comb_set) == c) and comb_set not in items_set: 
        items_set.append(comb_set) 
  return(items_set)

def apriori(baskets, support_value):
  print("Start apriori...")
  apriori_frequent=[]
  print("Create singleton...")
  singleton= baskets.flatMap(lambda line: line).map(lambda item: (item , 1)).reduceByKey(lambda a,b: (a+b))
  print("Complete creating singleton...")
  suppRdd= singleton.filter(lambda item: item[1] >= support_value)
  suppRdd= suppRdd.map(lambda item: (item[0]))
  #find the possible pairs  
  print("start creating possible_combinations")
  possible_combinations = list(combinations(suppRdd.toLocalIterator(), 2))
  print("possible_combinations created")
  #initialize support as empty rdd
  print("support start")
  support_rdd = sc.parallelize([])
  print("support done")
  baskets.cache()
  print("baskets cache done")
  c=2
  print("start while loop...")
  while (len(possible_combinations) > 0):
    #check if the possible items are frequents in the dataset
    frequent_items = baskets.flatMap(lambda cast: [(tuple(item), 1) for item in possible_combinations if set(list(item)).issubset(set(cast))]) \
                    .reduceByKey(lambda a, b: a+b) \
                    .filter(lambda item: item[1] >= support_value)
    print("frequent_items done")
    #add the frequent items and their occurrencies to support rdd
    support_rdd = support_rdd.union(frequent_items)
    print("support union done")
    #generates new items from the last frequent items found using the function defined above
    possible_combinations = getFrequentItemsets(frequent_items, c)
    print("getFrequentItemsets function done")
    c += 1
    #print("Len com: "+str(len(possible_combinations)))
    print("c: " + str(c))

  return support_rdd

In [ ]:
start_time = time.time()
aprioriResult = apriori(indexed_rdd, count*0.0003)
end_time = time.time()
apriori_time = end_time - start_time

Start apriori...
Create singleton...
Complete creating singleton...
start creating possible_combinations
possible_combinations created
support start
support done
baskets cache done
start while loop...
frequent_items done
support union done
Start function getFrequentItemsets
getFrequentItemsets candidateItemsets.map done
getFrequentItemsets singleton done
getFrequentItemsets product_iterator running...
getFrequentItemsets start for loop
getFrequentItemsets for comb length: 3
getFrequentItemsets for comb length: 3
getFrequentItemsets for comb length: 3
getFrequentItemsets for comb length: 3
getFrequentItemsets for comb length: 3
getFrequentItemsets for comb length: 3
getFrequentItemsets for comb length: 3
getFrequentItemsets for comb length: 3
getFrequentItemsets for comb length: 3
getFrequentItemsets for comb length: 3
getFrequentItemsets for comb length: 3
getFrequentItemsets for comb length: 3
getFrequentItemsets for comb length: 3
getFrequentItemsets for comb length: 3
getFrequentIte

In [ ]:
aprioriResult.collect()

[(('nm0623427', 'nm0006982'), 237),
 (('nm0006982', 'nm0619779'), 122),
 (('nm0006982', 'nm0419653'), 162),
 (('nm0046850', 'nm0006982'), 169),
 (('nm2082516', 'nm0648803'), 147),
 (('nm2373718', 'nm0648803'), 126),
 (('nm0006982', 'nm0046850'), 169),
 (('nm0648803', 'nm2082516'), 147),
 (('nm0006982', 'nm0619779'), 122),
 (('nm0006982', 'nm0623427'), 237),
 (('nm0648803', 'nm2373718'), 126),
 (('nm0006982', 'nm0419653'), 162)]

In [ ]:
end_time - start_time

# FP GROWTH

In [ ]:
from pyspark.ml.fpm import FPGrowth
start_time = time.time()
fp_growth = FPGrowth(itemsCol="items", minSupport= 0.0003)
fp_growth = fp_growth.fit(baskets)
end_time = time.time()

In [ ]:
exe_time = end_time - start_time
print(exe_time)

0.05721449851989746


In [ ]:
fp_growth.freqItemsets.show(10, False)

+-----------+----+
|items      |freq|
+-----------+----+
|[nm1388202]|153 |
|[nm0430646]|120 |
|[nm0103977]|798 |
|[nm0006982]|585 |
|[nm0436922]|152 |
|[nm0408381]|120 |
|[nm0648803]|565 |
|[nm0405977]|152 |
|[nm0576495]|120 |
|[nm0579663]|120 |
+-----------+----+
only showing top 10 rows



In [ ]:
fp_growth_freqItemsets = fp_growth.freqItemsets

In [ ]:
fp_growth_freqItemsets.filter("size(items) >= 2").show(10, False)

+----------------------+----+
|items                 |freq|
+----------------------+----+
|[nm0623427, nm0006982]|237 |
|[nm0046850, nm0006982]|169 |
|[nm0419653, nm0006982]|162 |
|[nm0619779, nm0006982]|122 |
|[nm2373718, nm0648803]|126 |
|[nm2082516, nm0648803]|147 |
+----------------------+----+



In [ ]:
ar = fp_growth.associationRules

In [ ]:
ar.show(10, False)

+-----------+-----------+------------------+-----------------+--------------------+
|antecedent |consequent |confidence        |lift             |support             |
+-----------+-----------+------------------+-----------------+--------------------+
|[nm2082516]|[nm0648803]|0.9130434782608695|636.1505194305502|3.734224805413864E-4|
|[nm2373718]|[nm0648803]|0.984375          |685.849778761062 |3.200764118926169E-4|
+-----------+-----------+------------------+-----------------+--------------------+



In [ ]:
count*0.0003

118.09679999999999

In [ ]:
baskets.count()

393656

In [ ]:
393656 * 0.0003

118.09679999999999